In [1]:
import numpy as np
import pandas as pd
import argparse
import os

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

from custom_dataset import CustomDataset
from network import Network
from utils import *

from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import roc_auc_score

In [2]:
parent_directory = '/data/users2/pnadigapusuresh1/JobOutputs'

In [3]:
########################
# Loading the Data #####
########################



torch.manual_seed(52)
# number of subprocesses to use for data loading
num_workers = 4
# how many samples per batch to load
batch_size = 15
if torch.cuda.device_count() > 1:
    batch_size *= torch.cuda.device_count()
else:
    batch_size = 5
# percentage of training set to use as validation
valid_size = 0.20
# percentage of data to be used for testset
test_size = 0.10


train_data = CustomDataset(transform = 
                        transforms.Compose([
                            transforms.RandomHorizontalFlip()
                            ]),train=True)

valid_data = CustomDataset(train=False,valid=True)

test_data = CustomDataset(train=False,valid=False)

# get filtered variables
vars = valid_data.vars

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,2))

model = nn.DataParallel(model)

load_path = os.path.join(parent_directory,'1504485','models_fold','2','epoch_45')

model.load_state_dict(torch.load(load_path))

Using cuda device


<All keys matched successfully>

In [5]:
    train_loader = DataLoader(train_data,batch_size=batch_size)

In [6]:
actual_valid= torch.tensor([]).to(device)
pred_valid = torch.tensor([]).to(device)
for X,y  in train_loader:
    X,y = X.to(device),y.to(device)
    actual_valid = torch.cat((actual_valid,y),0)
    pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
    soft_max = F.softmax(pred,dim=1)
    pred_proba_valid = torch.cat((pred_proba_valid,soft_max[:,-1]),0)
    correct = torch.eq(torch.max(soft_max, dim=1)[1],y).view(-1)
    pred_valid = torch.cat((pred_valid,torch.max(soft_max, dim=1)[1]),0)

RuntimeError: expected scalar type Double but found Float

In [7]:
actual_valid= torch.tensor([]).to(device)
pred_valid = torch.tensor([]).to(device)
for X,y  in train_loader:
    X,y = X.to(device),y.to(device)
    actual_valid = torch.cat((actual_valid,y.double()),0)
    pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
    soft_max = F.softmax(pred,dim=1)
    pred_proba_valid = torch.cat((pred_proba_valid,soft_max[:,-1]),0)
    correct = torch.eq(torch.max(soft_max, dim=1)[1],y).view(-1)
    pred_valid = torch.cat((pred_valid,torch.max(soft_max, dim=1)[1]),0)

RuntimeError: expected scalar type Double but found Float

In [8]:
actual_valid= torch.tensor([]).to(device)
pred_valid = torch.tensor([]).to(device)
for X,y  in train_loader:
    X,y = X.to(device),y.to(device)
    actual_valid = torch.cat((actual_valid,y.float()),0)
    pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
    soft_max = F.softmax(pred,dim=1)
    pred_proba_valid = torch.cat((pred_proba_valid,soft_max[:,-1]),0)
    correct = torch.eq(torch.max(soft_max, dim=1)[1],y).view(-1)
    pred_valid = torch.cat((pred_valid,torch.max(soft_max, dim=1)[1]),0)

NameError: name 'pred_proba_valid' is not defined

In [9]:
actual_valid= torch.tensor([]).to(device)
pred_valid = torch.tensor([]).to(device)
for X,y  in train_loader:
    X,y = X.to(device),y.to(device)
    actual_valid = torch.cat((actual_valid,y.float()),0)
    pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
    soft_max = F.softmax(pred,dim=1)
    correct = torch.eq(torch.max(soft_max, dim=1)[1],y).view(-1)
    pred_valid = torch.cat((pred_valid,torch.max(soft_max, dim=1)[1]),0)

In [10]:
torch.eq(actual_valid,pred_valid)

tensor([ True,  True,  True,  ...,  True, False, False], device='cuda:0')

In [11]:
sum(torch.eq(actual_valid,pred_valid))

tensor(3760, device='cuda:0')

In [12]:
3760/vars.shape[0]

0.7639171068671272

In [13]:
pred_valid.cpu().numpy()

array([0., 0., 1., ..., 0., 0., 1.], dtype=float32)

In [14]:
vars['pred'] = pred_valid.cpu().numpy()

In [15]:
vars

,sex,score,new_score,pred
eid,,,,
1001211,0.0,5.0,0,0.0
1002059,0.0,5.0,0,0.0
1002777,1.0,9.0,1,1.0
1002981,0.0,4.0,0,0.0
1003281,0.0,5.0,0,0.0
...,...,...,...,...
6019307,1.0,9.0,1,1.0
6021466,0.0,9.0,1,1.0
6023498,0.0,3.0,0,0.0


In [16]:
vars_f = vars.sex == 0

In [17]:
vars_f

eid
1001211     True
1002059     True
1002777    False
1002981     True
1003281     True
           ...  
6019307    False
6021466     True
6023498     True
6023725     True
6023996     True
Name: sex, Length: 4922, dtype: bool

In [18]:
vars_f = vars[vars.sex == 0]

In [19]:
vars_f

,sex,score,new_score,pred
eid,,,,
1001211,0.0,5.0,0,0.0
1002059,0.0,5.0,0,0.0
1002981,0.0,4.0,0,0.0
1003281,0.0,5.0,0,0.0
1006390,0.0,5.0,0,0.0
...,...,...,...,...
6019251,0.0,9.0,1,1.0
6021466,0.0,9.0,1,1.0
6023498,0.0,3.0,0,0.0


In [20]:
vars_m = vars[vars.sex == 1]

In [21]:
vars_m

,sex,score,new_score,pred
eid,,,,
1002777,1.0,9.0,1,1.0
1007754,1.0,9.0,1,0.0
1007834,1.0,9.0,1,1.0
1008148,1.0,9.0,1,1.0
1009028,1.0,11.0,1,0.0
...,...,...,...,...
6012232,1.0,5.0,0,0.0
6013814,1.0,9.0,1,1.0
6015965,1.0,9.0,1,1.0


In [22]:
2627 + 2295

4922

In [23]:
correct = vars_f[vars_f.new_score == vars_f.pred]

In [24]:
correct.shape

(2067, 4)

In [25]:
2067/2627

0.7868290826037305

In [26]:
correct = vars_m[vars_m.new_score == vars_m.pred]

In [27]:
correct.shape

(1693, 4)

In [28]:
1693/2295

0.7376906318082789